In [1]:
!pip install torch
!pip install torchvision

In [2]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn as nn
import torch.nn.functional as Fn
import matplotlib.pyplot as plt
import pandas as pd
from torch.utils.data import Dataset
from tqdm.notebook import tqdm

In [3]:
class CustomDataSet(Dataset):
    def __init__(self, csv_file):
        self.classes_k = {'Classe A':0,'Classe B':1, 'Classe C':2,'Classe D':3,'Classe E':4}
        self.df = pd.read_csv(csv_file)
        
    def __len__(self):
        return self.df.shape[0]

    def __getitem__(self, index):
        label = self.classes_k[self.df.loc[index]['Classe']]
        data = torch.tensor(self.df.loc[index][1:])
        return data, label


In [4]:
train_dataset = CustomDataSet("./dataset/processed/train_data.csv")
val_dataset = CustomDataSet("./dataset/processed/test_data.csv")

In [5]:
batch_size = 16
validation_split = .3
shuffle_dataset = True
random_seed= 42
num_epochs = 200
in_features = len(train_dataset[0][0])
out_classes = 5

In [6]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(val_dataset, batch_size=1, shuffle=True)

In [7]:
class SimpleMLP(nn.Module):
    def __init__(self, in_features, out_features=5):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(in_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, out_features)
        )

    def forward(self, x):
        return self.layers(x)
MLP_model = SimpleMLP(in_features).double()

In [8]:
learning_rate = 0.1
MLP_criterion = nn.CrossEntropyLoss()#BCEWithLogitsLoss
Model_optimizer = torch.optim.Adam(MLP_model.parameters())

In [9]:
train_losses = []
train_acc = []
val_loss = []
val_acc = []
best_loss = 100
for epoch in range(2): # monitoring the losses
    training_loss = 0
    validation_loss = 0
    train_correct = 0
    correct = 0
    MLP_model.train() ## here we are preparing the model for training
    for batch_idx, (data,label) in enumerate(train_loader):
        Model_optimizer.zero_grad() ## Clearing the gradients or we can say making gradients to zero of all the optimized variables
        output = MLP_model(data)
        loss = MLP_criterion(output,label) ## Calculating the loss
        loss.backward() ## backward pass, Computing the gradient of the loss w.r.t the parameters of the model
        Model_optimizer.step() ## a single step of optimization
        training_loss += loss.item() * data.size(0) ## updating the running training loss
        predicted_class = output.argmax(1)
        correct_ones = (predicted_class == label).type(torch.float)
        train_correct += correct_ones.sum().item()
        # if batch_idx%800 ==0:
        #     # print(output)
        #     print(f'Train Epoch: {epoch} b{batch_idx}\tLoss {training_loss/(data.size(0)*(batch_idx+1)):.6f}')
    print(f'train_correct = {100*train_correct/len(train_dataset)}')
    MLP_model.eval() ## here we are preparing the model for evaluation
    for data,label in validation_loader:
        output = MLP_model(data) ## forward passing as in this computing the predicted outputs by passing the inputs to the model
        loss = MLP_criterion(output,label) ## Calculating the loss
        validation_loss += loss.item() * data.size(0) ## updating the running validation loss
        predicted_class = output.argmax(1)
        correct_ones = (predicted_class == label).type(torch.float)
        correct += correct_ones.sum().item()
        # pred_y = torch.max(torch.exp(output),1)[1].data.cpu().numpy()
        # print(pred_y)
        # label_y = torch.max(label,1)[1].data.cpu().numpy()
        # correct += sum((pred_y == label_y))
    train_losses.append(training_loss/len(train_dataset))
    val_loss.append(validation_loss/len(val_dataset))
    train_acc.append(100*train_correct/len(train_dataset))
    val_acc.append(correct * 100 /len(val_dataset))
    if epoch%10==0:
        v_loss = validation_loss/len(val_dataset)
        print(f'e{epoch}: Train Loss: {training_loss/len(train_dataset):.04f}; Val Loss: {validation_loss/len(val_dataset):.04f} acc: {correct * 100 /len(val_dataset):.04f}')
        # torch.save({
        #     'epoch': epoch,
        #     'model_state_dict': MLP_model.state_dict(),
        #     'optimizer_state_dict': Model_optimizer.state_dict(),
        #     'loss': validation_loss/len(valid_sampler),
        #     }, f"./model_{epoch}.checkpoint")
        # if v_loss < best_loss:
        #     best_loss = v_loss
        #     torch.save({
        #         'epoch': epoch,
        #         'model_state_dict': MLP_model.state_dict(),
        #         'optimizer_state_dict': Model_optimizer.state_dict(),
        #         'loss': validation_loss/len(valid_sampler),
        #         }, f"./model_best.checkpoint")

/home/sheldon/anaconda3/envs/basics/lib/python3.11/site-packages/torch/autograd/__init__.py:266: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 9010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


train_correct = 80.81142857142858


NameError: name 'valid_sampler' is not defined